Data Wrangling

Forecasting Peer-to-Peer Lending Risk

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas.io.sql as pd_sql
import sqlite3 as sql

%matplotlib inline

Use pandas to open up the csv.

I have started with the archived file LoanStats3a_07_13 consisting of Lending Club loans from 2007-2013. To prepare the file for analysis, I removed four blank lines between 2007-2011 dataset and 2011-2013 datasets, removed three blank lines between 2007-2011 dataset and loans that did not meet policy guideline, and removed rows of text at the bottom of the combined dataset.

In [2]:
# Import the file into a dataframe
df = pd.read_csv("C:/Users/Steve/Georgetown/Capstone/data/Archived_data_files/LoanStats3a_07_13.csv", skiprows=1)

In [3]:
# Use pandas to view the first 5 rows.
pd.options.display.max_columns = 125
df.describe()

C:\Users\Steve\Anaconda2\lib\site-packages\numpy\lib\function_base.py:3834: RuntimeWarning: Invalid value encountered in percentile
  RuntimeWarning)


,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,total_acc,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_amnt,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_il_6m,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
count,2.306580e+05,2.306580e+05,230658.000000,230658.000000,230658.000000,230658.000000,2.306540e+05,230658.000000,230629.000000,230629.000000,96188.000000,21114.000000,230629.000000,230629.000000,2.306580e+05,230629.000000,230658.000000,230658.000000,230658.000000,230658.000000,230658.000000,230658.000000,230658.000000,230658.000000,230658.000000,230658.000000,230513.000000,32497.000000,230658.0,0.0,0.0,0.0,230629.000000,160382.000000,1.603820e+05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.603820e+05,0.0,0.0,0.0,180628.000000,160376.000000,179099.000000,179012.000000,230513.000000,230629.000000,154252.000000,160381.000000,160381.000000,160382.000000,180628.000000,179296.000000,36735.000000,160263.000000,54426.000000,160382.000000,160382.000000,160382.000000,172068.000000,160382.000000,160382.000000,160382.000000,160382.000000,160382.000000,172068.000000,160126.000000,160382.000000,160382.000000,160382.000000,160229.000000,179096.000000,229293.000000,230553.000000,1.603820e+05,1.806280e+05,180628.000000,1.603820e+05
mean,4.151386e+06,4.973037e+06,13752.488966,13700.987067,13565.474716,421.415659,7.166669e+04,16.379008,0.223563,0.854788,34.983407,81.375296,10.695598,0.097325,1.594624e+04,24.097520,1139.019607,1138.306455,14334.273851,14193.998797,11218.060405,2976.573350,0.931846,138.708262,14.233710,3638.134300,0.002590,41.792473,1.0,NaN,NaN,NaN,0.002233,76.746518,1.373305e+05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.989411e+04,NaN,NaN,NaN,3.929529,13796.625106,8264.036231,66.829415,0.004299,6.853358,125.076498,178.482058,14.108305,8.938734,1.811048,25.658057,40.816850,6.991770,36.609947,0.329476,3.755272,5.675450,4.665993,9.018992,7.726397,8.094624,14.950375,5.693476,11.085617,0.000475,0.002014,0.063754,1.787975,95.399550,53.557030,0.077656,0.011455,1.655544e+05,4.288507e+04,20240.250448,3.439085e+04
std,3.008395e+06,3.608537e+06,8089.573944,8060.727753,8101.843886,241.413823,5.431335e+04,7.580052,0.673448,1.145318,21.744923,32.640114,4.632143,0.382257,1.983643e+04,11.208001,3288.004743,3286.338272,9318.828348,9339.127533,7519.994160,3048.575675,6.500591,707.488590,110.971405,5594.499934,0.053487,20.997645,0.0,NaN,NaN,NaN,0.052919,859.800234,1.507587e+05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.777504e+04,NaN,NaN,NaN,2.657120,16383.435302,13177.267513,26.110808,0.073837,474.350386,50.852168,88.084678,16.149825,9.695798,2.193365,29.483506,21.687196,5.880568,21.740286,0.952576,2.079838,2.919011,2.486306,4.808284,6.578756,3.887839,7.384420,2.923220,4.622759,0.022899,0.049334,0.367624,1.516782,7.391987,34.148464,0.277691,0.218258,1.672593e+05,4.018159e+04,18885.232505,3.688284e+04
min,5.473400e+04,7.047300e+04,500.000000,500.000000,0.

In [4]:
# Need to verify that no loan ids are null
a = df['id'].isnull().sum()
print (a)

0


In [5]:
# Also need to verify that there are no duplicate loan entries
df['id'].nunique()

230658

In [6]:
# Determine the number of columns in the dataframe.
print(len(df.columns))

111


In [7]:
# Althought lending club provides 111 features for each loan after they are funded, they provide a much small number of
# features to potential investors. Since we are developing a tool for investors, all features not provided to investors
# should be eliminated except those that reference the loan outcome: 'loan_status' and 'total_rec_prncp'
df = df.drop(['id', 'member_id', 'total_acc', 'initial_list_status', 'out_prncp', 'out_prncp_inv', 'total_pymnt', 
              'total_pymnt_inv', 'total_rec_int', 'total_rec_late_fee', 'recoveries',
              'collection_recovery_fee', 'last_pymnt_d', 'last_pymnt_amnt', 'next_pymnt_d', 'last_credit_pull_d', 
              'mths_since_last_major_derog', 'policy_code', 'application_type', 'annual_inc_joint', 'dti_joint',
              'verification_status_joint', 'tot_coll_amt', 'tot_cur_bal', 'open_acc_6m', 'open_il_6m', 'open_il_12m',
              'open_il_24m', 'mths_since_rcnt_il', 'total_bal_il', 'il_util', 'open_rv_12m', 'open_rv_24m', 'max_bal_bc',
              'all_util', 'total_rev_hi_lim', 'inq_fi', 'total_cu_tl', 'inq_last_12m', 'acc_open_past_24mths', 'avg_cur_bal',
              'bc_open_to_buy', 'bc_util', 'chargeoff_within_12_mths', 'mo_sin_old_il_acct', 'mo_sin_old_rev_tl_op', 
              'mo_sin_rcnt_rev_tl_op', 'mo_sin_rcnt_tl', 'mort_acc', 'mths_since_recent_bc', 'mths_since_recent_bc_dlq', 
              'mths_since_recent_inq', 'mths_since_recent_revol_delinq', 'num_accts_ever_120_pd', 'num_actv_bc_tl', 
              'num_actv_rev_tl', 'num_bc_sats', 'num_bc_tl', 'num_il_tl', 'num_op_rev_tl', 'num_rev_accts',  
              'num_rev_tl_bal_gt_0', 'num_sats', 'num_tl_120dpd_2m', 'num_tl_30dpd', 'num_tl_90g_dpd_24m', 'num_tl_op_past_12m', 
              'pct_tl_nvr_dlq', 'percent_bc_gt_75', 'pub_rec_bankruptcies', 'tax_liens', 'tot_hi_cred_lim', 'total_bal_ex_mort', 
              'total_bc_limit', 'total_il_high_credit_limit'], axis = 1)

Some additional columns can be eliminated.

- loan_amnt; loan_amnt and funded_amnt are redundant
- funded_amount_inv; the difference between funded_amnt and funded_amnt_inv is the amount invested by Lending Club and this is      not germane to predicting default
- url; provides no additonal useful info
- desc; free form entry
- title; free form entry
- emp_title; too many different entries to be useful
- payment_plan; only relevent after loan has been issued
- zip_code; we will not have zip code level data

In [8]:
df = df.drop(['loan_amnt', 'funded_amnt_inv', 'url', 'desc', 'title', 'emp_title', 'pymnt_plan', 'zip_code'], axis = 1)

In [9]:
# Since we ony want to consider completed loans, can further reduce the size of the dataframe by eliminating loans that
# originated within the last 36 months
df = df [df.issue_d != '13-Dec']
df = df [df.issue_d != '13-Nov']
df = df [df.issue_d != '13-Oct']
df = df [df.issue_d != '13-Sep']
df = df [df.issue_d != '13-Aug']
df = df [df.issue_d != '13-Jul']
df = df [df.issue_d != '13-Jun']
df = df [df.issue_d != '13-May']

In [10]:
# Useful to reset id's at this point
df = df.reset_index(drop=True)

In [11]:
# Need to drop 60 month loans that originated within the last 60 months
# Determine rows in dataframe
length = len(df.index)
print(length)

128027


In [12]:
for i in range (0,128027):    
    if (df.loc[i,'term'] == ' 60 months'):
        if (df.loc[i,'issue_d'] == '13-Apr' or df.loc[i,'issue_d'] == '13-Mar' or df.loc[i,'issue_d'] == '13-Feb' 
            or df.loc[i,'issue_d'] == '13-Jan' or df.loc[i,'issue_d'] == '12-Dec' or df.loc[i,'issue_d'] == '12-Nov'  
            or df.loc[i,'issue_d'] == '12-Oct' or df.loc[i,'issue_d'] == '12-Sep' or df.loc[i,'issue_d'] == '12-Aug' 
            or df.loc[i,'issue_d'] == '12-Jul' or df.loc[i,'issue_d'] == '12-Jun' or df.loc[i,'issue_d'] == '12-May' 
            or df.loc[i,'issue_d'] == '12-Apr' or df.loc[i,'issue_d'] == '12-Mar' or df.loc[i,'issue_d'] == '12-Feb' 
            or df.loc[i,'issue_d'] == '12-Jan' or df.loc[i,'issue_d'] == '11-Dec' or df.loc[i,'issue_d'] == '11-Nov'  
            or df.loc[i,'issue_d'] == '11-Oct' or df.loc[i,'issue_d'] == '11-Sep' or df.loc[i,'issue_d'] == '11-Aug' 
            or df.loc[i,'issue_d'] == '11-Jul' or df.loc[i,'issue_d'] == '11-Jun' or df.loc[i,'issue_d'] == '11-May'
           ):
            df = df.drop(i,axis=0)

In [13]:
df.head(n=5)

,funded_amnt,term,int_rate,installment,grade,sub_grade,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,purpose,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_rec_prncp,collections_12_mths_ex_med,acc_now_delinq,delinq_amnt
0,4625,36 months,8.90%,146.86,A,A5,7 years,RENT,43000.0,Verified,13-Apr,Fully Paid,credit_card,FL,7.42,0.0,3-Oct,0.0,NaN,NaN,6.0,0.0,4659,25.60%,4625.0,0.0,0.0,0.0
1,21250,36 months,8.90%,674.76,A,A5,10+ years,MORTGAGE,47500.0,Verified,13-Apr,Fully Paid,debt_consolidation,PA,17.58,0.0,Jun-94,0.0,67.0,NaN,10.0,0.0,9637,33.30%,21250.0,0.0,0.0,0.0
3,12000,36 months,12.12%,399.26,B,B3,10+ years,MORTGAGE,68000.0,Not Verified,13-Apr,Fully Paid,credit_card,NY,9.99,0.0,5-Aug,0.0,NaN,NaN,13.0,0.0,17458,61.30%,12000.0,0.0,0.0,0.0
4,14000,36 months,7.90%,438.07,A,A4,10+ years,MORTGAGE,75900.0,Not Verified,13-Apr,Fully Paid,debt_consolidation,MA,19.11,0.0,Nov-00,0.0,NaN,NaN,10.0,0.0,30512,63.30%,14000.0,0.0,0.0,0.0
5,10000,36 months,8.90%,317.54,A,A5,10+ years,MORTGAGE,80000.0,Source Verified,13-Apr,Fully Paid,home_improvement,CA,12.17,0.0,Oct-00,0.0,NaN,NaN,11.0,0.0,30196,74.40%,10000.0,0.0,0.0,0.0


In [14]:
# Determine rows in dataframe
length = len(df.index)
print(length)

105736


In [15]:
# Useful to reset id's at this point
df = df.reset_index(drop=True)

In [16]:
# Because our objective is to forecast probability of default and impact of default, we need to more thoroughly understand the 
# column 'loan_status'
df['loan_status'].value_counts()

Fully Paid                                             89509
Charged Off                                            13350
Does not meet the credit policy. Status:Fully Paid      1988
Does not meet the credit policy. Status:Charged Off      761
Late (31-120 days)                                        81
Default                                                   23
Current                                                   21
In Grace Period                                            2
Late (16-30 days)                                          1
Name: loan_status, dtype: int64

In [17]:
# Consolidate the first four categories into either: 'Paid' or 'Default'
for i in range (0,105736):
    if df.loc[i, 'loan_status'] == 'Fully Paid':
        df.loc[i, 'loan_status'] = 'Paid'
    elif df.loc[i, 'loan_status'] == 'Does not meet the credit policy. Status:Fully Paid':
        df.loc[i, 'loan_status'] = 'Paid'
    elif df.loc[i, 'loan_status'] == 'Does not meet the credit policy. Status:Charged Off':
        df.loc[i, 'loan_status'] = 'Default'
    elif df.loc[i, 'loan_status'] == 'Charged Off':
        df.loc[i, 'loan_status'] = 'Default'

In [18]:
# Let's see where the remaining four categories are showing up
dfexplore = df.loc[df['loan_status'].isin(['Late (31-120 days)', 'Current', 'In Grace Period', 'Late (16-30 days)'])]
pd.options.display.max_rows = 125
dfexplore.head(n=105)

,funded_amnt,term,int_rate,installment,grade,sub_grade,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,purpose,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_rec_prncp,collections_12_mths_ex_med,acc_now_delinq,delinq_amnt
154,14750,36 months,14.33%,506.49,C,C1,10+ years,MORTGAGE,110000.00,Verified,13-Apr,Current,debt_consolidation,GA,10.23,0.0,Jun-90,4.0,51.0,NaN,18.0,0.0,15463,48.60%,14745.00,0.0,0.0,0.0
160,11000,36 months,17.77%,396.41,D,D1,10+ years,RENT,55000.00,Verified,13-Apr,Late (31-120 days),debt_consolidation,VA,31.88,0.0,Dec-99,1.0,81.0,NaN,19.0,0.0,10980,49.20%,9844.88,0.0,0.0,0.0
233,10000,36 months,13.11%,337.47,B,B4,4 years,MORTGAGE,79941.00,Not Verified,13-Apr,Late (31-120 days),debt_consolidation,MI,16.87,0.0,Mar-93,1.0,NaN,NaN,10.0,0.0,22080,58.40%,9627.47,0.0,0.0,0.0
302,15000,36 months,18.75%,547.95,D,D3,10+ years,RENT,40000.00,Not Verified,13-Apr,Late (31-120 days),credit_card,KY,23.01,1.0,Jan-95,0.0,12.0,107.0,6.0,1.0,13782,67.90%,13035.13,0.0,0.0,0.0
318,8000,36 months,16.29%,282.41,C,C4,10+ years,RENT,45000.00,Not Verified,13-Apr,Late (31-120 days),credit_card,CA,16.32,0.0,May-99,0.0,NaN,NaN,7.0,0.0,8375,90.10%,7720.70,0.0,0.0,0.0
332,10800,36 months,14.33%,370.86,C,C1,10+ years,RENT,42000.00,Verified,13-Apr,Current,debt_consolidation,NJ,17.09,1.0,Jul-97,0.0,15.0,NaN,15.0,0.0,9303,41.70%,10792.30,0.0,0.0,0.0
426,12800,36 months,13.11%,431.97,B,B4,5 years,RENT,74899.81,Verified,13-Apr,Current,debt_consolidation,CA,16.68,0.0,Sep-00,0.0,66.0,NaN,9.0,0.0,14317,84.20%,12787.78,0.0,0.0,0.0
457,1000,36 months,19.05%,36.69,D,D4,4 years,RENT,66500.00,Not Verified,13-Apr,Late (31-120 days),debt_consolidation,CA,18.90,0.0,Oct-00,0.0,NaN,NaN,13.0,0.0,16328,92.80%,923.32,0.0,0.0,0.0
574,9000,36 months,16.29%,317.71,C,C4,10+ years,MORTGAGE,60964.80,Not Verified,13-Apr,Current,other,CA,22.52,0.0,Sep-99,0.0,55.0,NaN,11.0,0.0,22867,28%,8996.44,0.0,0.0,0.0
696,10000,36 months,12.12%,332.72,B,B3,7 years,RENT,32000.00,Not Verified,13-Apr,Current,debt_consolidation,AZ,14.63,0.0,Dec-98,0.0,NaN,105.0,14.0,1.0,12814,29.20%,9996.42,0.0,0.0,0.0


In [19]:
# Based on above, delete rows from 4/15/2013, 3/15/2013, and 2/15/2013.
df = df [df.issue_d != '13-Apr']
df = df [df.issue_d != '13-Mar']
df = df [df.issue_d != '13-Feb']

In [20]:
# Determine rows in dataframe
length = len(df.index)
print(length)
# Useful to reset id's at this point
df = df.reset_index(drop=True)

86047


In [21]:
# And delete 60 month loans after 02/15/11
global i
for i in range (0,86047):
    if df.loc[i,'term'] == ' 60 months':
        if df.loc[i,'issue_d'] == '11-Apr':
            df = df.drop(i,axis=0)
        elif df.loc[i,'issue_d'] == '11-Mar':
            df = df.drop(i,axis=0)

In [22]:
df['loan_status'].value_counts()

Paid                  73380
Default               11430
Late (31-120 days)       10
Name: loan_status, dtype: int64

In [23]:
# Will assume that 10 remaining loans that are late 31-120 days eventually default with no additional principal payments
df = df.reset_index(drop=True)
global i
for i in range (0,84820):
    if df.loc[i,'loan_status'] == 'Late (31-120 days)':
            df.loc[i,'loan_status'] = 'Default'

Let's explore the dataset a bit more

In [24]:
# Let's make sure there are no blanks in the state data (addr_state) column since we will be using addr_state to merge
# state level macroeconomic data
df['addr_state'].isnull().sum()

0

In [25]:
# Do we need to include the U.S. Virgin Islands and Puerto Rico?
a =df['addr_state'].unique()
print(a)

['WA' 'NY' 'VT' 'CA' 'IL' 'CO' 'TX' 'PA' 'OH' 'AZ' 'DC' 'FL' 'WI' 'UT' 'NV'
 'NC' 'AL' 'MD' 'IN' 'LA' 'NJ' 'CT' 'VA' 'SC' 'AK' 'OK' 'OR' 'GA' 'MA' 'AR'
 'MO' 'KS' 'MN' 'MI' 'NH' 'WV' 'DE' 'RI' 'KY' 'MT' 'NM' 'HI' 'SD' 'WY' 'TN'
 'MS' 'IA' 'NE' 'ID' 'ME']


No need to collect macroeconomic data for Virgin Islands or Puerto Rico

In [26]:
# Need to verify that there are no null values for feature 'term'
a = df['term'].isnull().sum()
print (a)

0


In [27]:
# Is all of the Lending Club data verified?
df['verification_status'].value_counts()

Not Verified       39394
Verified           26129
Source Verified    19297
Name: verification_status, dtype: int64

It is surprising that the majority of the loans are not verified. This calls data quality into question.

Let's start to clean up the dataset in anticipation of machine learning.

In [28]:
# Need to eliminate 'months' from column 'term'
for i in range (0,84820):
    if df.loc[i,'term'] == ' 36 months':
        df.loc[i,'term'] = 36
    if df.loc[i,'term'] == ' 60 months':
        df.loc[i,'term'] = 60

In [29]:
# Need to remove percent from int_rate and revol_util
for i in range (0,84820):
    df.loc[i,'int_rate'] = df.loc[i, 'int_rate'].rstrip('%')
    df.loc[i,'revol_util'] = float(str(df.loc[i, 'revol_util']).rstrip('%'))

In [30]:
# To be able to use datetime, need to add zero in front of years 7, 8, and 9
for i in range (0, 84820):
    s = df.loc[i,'issue_d']
    if (s[0]) != '1':
        s = '0' + s
        df.loc[i,'issue_d'] = s

In [31]:
# Need to convert issue_d to a format that will allow merging the economic data
import datetime
df['issue_d'] = list(map(lambda x: datetime.datetime.strptime(x, '%y-%b').strftime('%m/15/%y'), df['issue_d']))

It would be useful to create a column that captures the net loss of principal incurred by investors from loan defaults:

    'net_default_amnt' = 'funded_amnt' - 'total_rec_prncp'


In [32]:
# Create the column 'net_default_amnt' and initially fill with zeros
df['net_default_amnt'] = 0
# Fill the column with calculated values
df['net_default_amnt']= df['funded_amnt']- df['total_rec_prncp']

In [33]:
# Let's examine the 'emp_length' column.
df['emp_length'].value_counts()

10+ years    20872
2 years       8800
< 1 year      8418
3 years       7491
5 years       7053
4 years       6773
1 year        6615
6 years       5395
7 years       4311
8 years       3435
n/a           2861
9 years       2796
Name: emp_length, dtype: int64

Regarding 'emp_length' column, need to remove 'year' and 'years' as well as address '10+', '< 1',   and 'n/a' entries

In [34]:
# Need to strip 'years' and 'year' from right side of emp_length
for i in range (0,84820):
    df.loc[i,'emp_length'] = df.loc[i, 'emp_length'].rstrip('years')
    df.loc[i,'emp_length'] = df.loc[i, 'emp_length'].rstrip('year')

In [35]:
# Decided to:
#   - Set 10+ equal to 15 based on an extrapolation of a plot of number of borrowers versus emp_length
#   - Set < 1 equal to 0.5
#   - 'n/a' entries are now 'n/' since we removed a above -- will replace with 'NaN' for now 
global i
for i in range (0,84820):
    if df.loc[i, 'emp_length'] == '10+ ':
        df.loc[i, 'emp_length'] = '15'
    elif df.loc[i, 'emp_length'] == '< 1 ':
        df.loc[i, 'emp_length'] = '0.5'
    elif df.loc[i, 'emp_length'] == 'n/':
        df.loc[i, 'emp_length'] = 'NaN'

In [36]:
# Determine the emp_length mean
m = df['emp_length'].median()
print(m)

5.0


In [37]:
# Let's look as some of the lenders with 'NaN' to see if they appear to be working. If so, will set NaN equal to the median;
# otherwise will set NaN equal to 0 
dfempl = df.loc[df['emp_length'] == 'NaN']
dfempl.head(n=5)

,funded_amnt,term,int_rate,installment,grade,sub_grade,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,purpose,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_rec_prncp,collections_12_mths_ex_med,acc_now_delinq,delinq_amnt,net_default_amnt
77,15000,36,15.80,525.88,C,C3,NaN,RENT,40603.0,Verified,01/15/13,Paid,credit_card,NC,31.11,0.0,Feb-00,0.0,NaN,NaN,6.0,0.0,17412,97.8,15000.0,0.0,0.0,0.0,0.0
321,4800,36,21.00,180.85,E,E2,NaN,MORTGAGE,16152.0,Verified,01/15/13,Paid,debt_consolidation,MI,17.01,0.0,4-Jan,0.0,NaN,NaN,6.0,0.0,3233,67.4,4800.0,0.0,0.0,0.0,0.0
325,2200,36,15.31,76.60,C,C2,NaN,RENT,57000.0,Verified,01/15/13,Paid,debt_consolidation,CA,28.84,0.0,Jan-88,0.0,NaN,NaN,13.0,0.0,26552,87.9,2200.0,0.0,0.0,0.0,0.0
333,10625,36,16.29,375.07,C,C4,NaN,RENT,39600.0,Source Verified,01/15/13,Paid,debt_consolidation,FL,12.30,0.0,Jan-84,0.0,40.0,NaN,10.0,0.0,4406,68.8,10625.0,0.0,0.0,0.0,0.0
354,6500,36,13.11,219.36,B,B4,NaN,MORTGAGE,41000.0,Source Verified,01/15/13,Default,debt_consolidation,FL,31.69,0.0,Dec-65,0.0,77.0,NaN,15.0,0.0,33842,93,2749.3,0.0,0.0,0.0,3750.7


In [38]:
# Since borrowers with NaN have annual incomes, they appear to be working. Wii therefore replace NaN with median 5.0
df.loc[df['emp_length'] == 'NaN', 'emp_length'] = 5.0

In [39]:
# Let's check the values of the 'home_ownership' feature.
df['home_ownership'].value_counts()

RENT        40738
MORTGAGE    37124
OWN          6740
OTHER         175
NONE           43
Name: home_ownership, dtype: int64

In [40]:
# Based on the above, there are a small number of cells with the value 'None'. Will change these to 'Other'
df.loc[df['home_ownership'] == 'NONE', 'home_ownership'] = 'OTHER'

In [41]:
#See how many NaN values are in each column
df.isnull().sum()

funded_amnt                       0
term                              0
int_rate                          0
installment                       0
grade                             0
sub_grade                         0
emp_length                        0
home_ownership                    0
annual_inc                        4
verification_status               0
issue_d                           0
loan_status                       0
purpose                           0
addr_state                        0
dti                               0
delinq_2yrs                      29
earliest_cr_line                 29
inq_last_6mths                   29
mths_since_last_delinq        51296
mths_since_last_record        79273
open_acc                         29
pub_rec                          29
revol_bal                         0
revol_util                      131
total_rec_prncp                   0
collections_12_mths_ex_med      145
acc_now_delinq                   29
delinq_amnt                 

In [42]:
# 'mths_since_last_delinq' and 'mths_since_last_record' have too many NaN's to be useful. These columns should be deleted.
del df['mths_since_last_delinq']
del df['mths_since_last_record']

In [43]:
# Let's look at the rows where annual_inc is NaN
df.loc[df['annual_inc'].isnull()]

,funded_amnt,term,int_rate,installment,grade,sub_grade,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,purpose,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_rec_prncp,collections_12_mths_ex_med,acc_now_delinq,delinq_amnt,net_default_amnt
84734,5000,36,7.43,155.38,A,A2,0.5,OTHER,NaN,Not Verified,08/15/07,Paid,other,NY,1.0,NaN,NaN,NaN,NaN,NaN,0,NaN,5000.0,NaN,NaN,NaN,0.0
84735,7000,36,7.75,218.55,A,A3,0.5,OTHER,NaN,Not Verified,08/15/07,Paid,other,NY,1.0,NaN,NaN,NaN,NaN,NaN,0,NaN,7000.0,NaN,NaN,NaN,0.0
84765,6700,36,7.75,209.18,A,A3,0.5,OTHER,NaN,Not Verified,07/15/07,Paid,other,NY,1.0,NaN,NaN,NaN,NaN,NaN,0,NaN,6700.0,NaN,NaN,NaN,0.0
84818,6500,36,8.38,204.84,A,A5,0.5,OTHER,NaN,Not Verified,06/15/07,Paid,other,NY,4.0,NaN,NaN,NaN,NaN,NaN,0,NaN,6500.0,NaN,NaN,NaN,0.0


In [44]:
# Let's delete the above four rows since several of the values in the columns are NaN and also all of these loans ere paid off
df.drop(84734, axis = 0, inplace = True)
df.drop(84735, axis = 0, inplace = True)
df.drop(84765, axis = 0, inplace = True)
df.drop(84818, axis = 0, inplace = True)

In [45]:
# Let's look at annual_inc outliers
df.loc[df['annual_inc'] > 1000000.0]

,funded_amnt,term,int_rate,installment,grade,sub_grade,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,purpose,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_rec_prncp,collections_12_mths_ex_med,acc_now_delinq,delinq_amnt,net_default_amnt
5078,18000,36,13.11,607.45,B,B4,7,MORTGAGE,1350000.0,Source Verified,01/15/13,Paid,debt_consolidation,KS,1.38,0.0,Jul-00,5.0,10.0,0.0,17148,50.1,18000.0,0.0,0.0,0.0,0.0
5466,14825,36,13.11,500.30,B,B4,15,MORTGAGE,7141778.0,Source Verified,12/15/12,Paid,debt_consolidation,GA,0.25,0.0,Mar-96,2.0,17.0,0.0,11351,70.9,14825.0,0.0,0.0,0.0,0.0
6654,30000,36,11.14,984.16,B,B2,6,OWN,1200000.0,Source Verified,12/15/12,Paid,debt_consolidation,NY,2.18,0.0,4-Apr,0.0,16.0,0.0,81138,78.2,30000.0,0.0,0.0,0.0,0.0
7772,35000,36,15.31,1218.61,C,C2,15,MORTGAGE,5000000.0,Verified,12/15/12,Paid,credit_card,NY,2.36,1.0,Apr-94,2.0,13.0,0.0,975800,0,35000.0,0.0,0.0,0.0,0.0
11348,20000,36,8.90,635.07,A,A5,3,MORTGAGE,1250000.0,Source Verified,11/15/12,Paid,home_improvement,HI,2.01,0.0,Oct-92,2.0,19.0,0.0,22427,31.7,20000.0,0.0,0.0,0.0,0.0
22340,35000,36,12.12,1164.51,B,B3,5,MORTGAGE,1200000.0,Source Verified,09/15/12,Paid,other,NY,0.35,0.0,Jan-87,2.0,5.0,0.0,20228,47.8,35000.0,0.0,0.0,0.0,0.0
37094,33000,36,7.62,1028.33,A,A3,15,MORTGAGE,1233000.0,Verified,06/15/12,Paid,debt_consolidation,NY,0.26,0.0,Apr-88,0.0,7.0,0.0,95435,1.9,33000.0,0.0,0.0,0.0,0.0
53867,12025,36,14.27,412.57,C,C2,15,RENT,1782000.0,Not Verified,09/15/11,Paid,debt_consolidation,NY,1.31,0.0,Dec-87,0.0,9.0,0.0,49208,85.7,12025.0,0.0,0.0,0.0,0.0
63762,10000,36,6.54,306.68,A,A4,15,OWN,1200000.0,Verified,01/15/11,Paid,debt_consolidation,CT,1.46,0.0,Jun-94,0.0,6.0,0.0,75706,49.1,10000.0,0.0,0.0,0.0,0.0
65903,1500,36,6.17,45.75,A,A3,5,MORTGAGE,1900000.0,Source Verified,11/15/10,Paid,credit_card,TX,0.57,0.0,Jun-81,2.0,17.0,0.0,17396,59.6,1500.0,0.0,0.0,0.0,0.0


In [46]:
# I will delete the above since:
# - it is unclear if they are real or errors
# - all of these loans were paid off and hence deleting them should only make results more conservative (overpredict defaults)
df = df.drop(df[df.annual_inc > 1000000].index)

In [47]:
# 'delinq_2yrs has 29 NaN values
df['delinq_2yrs'].value_counts()

0.0     74151
1.0      7815
2.0      1852
3.0       569
4.0       186
5.0        97
6.0        41
7.0        27
9.0         8
8.0         8
11.0        6
10.0        5
13.0        2
18.0        1
14.0        1
12.0        1
Name: delinq_2yrs, dtype: int64

In [48]:
# Based on the above, I would retain this column and set the NaN values equal to the mean
m = df['delinq_2yrs'].median()
df['delinq_2yrs'].fillna(value=m, inplace = True)

In [49]:
df['inq_last_6mths'].value_counts()

0.0     41222
1.0     23090
2.0     11707
3.0      5947
4.0      1310
5.0       672
6.0       379
7.0       183
8.0       115
9.0        50
10.0       24
12.0       15
11.0       15
15.0        9
13.0        6
14.0        6
18.0        4
16.0        3
17.0        2
19.0        2
24.0        2
32.0        1
20.0        1
25.0        1
27.0        1
28.0        1
31.0        1
33.0        1
Name: inq_last_6mths, dtype: int64

In [50]:
# Based on the above, I would retain this column and set the NaN values equal to the mean
m = df['inq_last_6mths'].median()
df['inq_last_6mths'].fillna(value=m, inplace = True)

In [51]:
df['open_acc'].value_counts()

8.0     8229
9.0     8196
7.0     7935
6.0     7354
10.0    7315
11.0    6573
12.0    5701
5.0     5397
13.0    4669
14.0    3700
4.0     3680
15.0    3054
16.0    2463
3.0     2114
17.0    1886
18.0    1477
19.0    1105
20.0     890
2.0      856
21.0     686
22.0     435
23.0     290
24.0     228
25.0     151
26.0      83
27.0      64
28.0      50
1.0       42
29.0      28
31.0      26
30.0      22
34.0      14
32.0      11
38.0       8
33.0       7
35.0       6
36.0       6
37.0       6
39.0       2
44.0       2
46.0       2
52.0       1
49.0       1
41.0       1
42.0       1
45.0       1
47.0       1
0.0        1
Name: open_acc, dtype: int64

In [52]:
# Based on the above, I would retain this column and set the NaN values equal to the mean
m = df['open_acc'].median()
df['open_acc'].fillna(value=m, inplace = True)

In [53]:
df['pub_rec'].value_counts()

0.0    80498
1.0     4038
2.0      178
3.0       37
4.0       10
5.0        6
8.0        1
7.0        1
6.0        1
Name: pub_rec, dtype: int64

In [54]:
# Based on the above, I would retain this column and set the NaN values equal to the mean
m = df['pub_rec'].median()
df['pub_rec'].fillna(value=m, inplace = True)

In [55]:
df['revol_util'].value_counts()

0.00      1154
55.50      138
55.80      137
80.60      137
63.00      136
62.00      135
57.40      134
64.30      133
64.90      133
68.50      131
76.20      130
66.50      129
64.70      129
64.60      129
70.10      128
47.60      128
62.80      127
54.10      127
61.50      127
81.60      127
57.50      126
64.50      126
71.50      126
65.40      125
62.60      125
63.90      125
69.40      125
71.60      125
66.60      124
75.70      124
48.90      124
61.30      124
84.00      123
76.90      123
70.40      123
72.40      123
67.70      123
68.70      123
63.70      123
66.70      123
59.20      122
61.00      122
58.00      122
75.50      122
67.30      122
64.10      121
70.00      121
54.00      121
82.90      121
58.80      121
60.80      121
65.00      120
65.90      120
76.60      120
58.30      120
58.20      120
65.20      120
73.70      120
69.50      120
69.60      120
70.80      120
81.40      119
          ... 
21.92        1
36.94        1
27.81        1
23.11     

In [56]:
# I would retain this column and set the NaN values equal to the mean
m = df['revol_util'].median()
df['revol_util'].fillna(value=m, inplace = True)

In [57]:
df['collections_12_mths_ex_med'].value_counts()

0.0    84621
1.0       31
2.0        2
Name: collections_12_mths_ex_med, dtype: int64

In [58]:
# Since there are only 33 entries that are not zero, this feature could not explain the 10+K defaults and I would delete it
del df['collections_12_mths_ex_med']

In [59]:
df['acc_now_delinq'].value_counts()

0.0    84748
1.0       21
2.0        1
Name: acc_now_delinq, dtype: int64

In [60]:
# Since there are only 22 entries that are not zero, this feature could not explain the 10+K defaults and I would delete it
del df['acc_now_delinq']

In [61]:
df['delinq_amnt'].value_counts()

0.0        84752
2378.0         1
27.0           1
48.0           1
404.0          1
440.0          1
1061.0         1
1164.0         1
1380.0         1
65000.0        1
31219.0        1
6053.0         1
6855.0         1
12641.0        1
13063.0        1
19028.0        1
21059.0        1
24521.0        1
4246.0         1
Name: delinq_amnt, dtype: int64

In [62]:
# Since there are only 18 entries that are not zero, this feature could not explain the 10+K defaults and I would delete it
del df['delinq_amnt']

In [63]:
# I will delete the column 'earliest_cr_line' since it likely cannot be verified and formatting is ambiguous
del df['earliest_cr_line']

In [64]:
# Check to confirm there are no NaN values in any column
df.isnull().sum()

funded_amnt            0
term                   0
int_rate               0
installment            0
grade                  0
sub_grade              0
emp_length             0
home_ownership         0
annual_inc             0
verification_status    0
issue_d                0
loan_status            0
purpose                0
addr_state             0
dti                    0
delinq_2yrs            0
inq_last_6mths         0
open_acc               0
pub_rec                0
revol_bal              0
revol_util             0
total_rec_prncp        0
net_default_amnt       0
dtype: int64

In [65]:
# Let's save df dataframe
df.to_csv ('C:/Users/Steve/Georgetown/Capstone/data/Archived_data_files/dfsave.csv', sep=',')

Let's import macroeconomic data

In [66]:
# Need to import macro economic data files
# df_unemp provide monthly state level unemployment data, including slope of 12-month prior least square fit
df_unemp = pd.read_csv("C:/Users/Steve/Georgetown/Capstone/data/Monthly_Data_12m_prior/F_unemp_state_monthly.csv", skiprows=0)

In [67]:
df_unemp.tail(n=3)

,addr_state,issue_d,unemp,unemp_LSFT
6422,WY,04/15/16,0.058,0.001747
6423,WY,05/15/16,0.054,0.001830
6424,WY,06/15/16,0.056,0.001907


In [68]:
# df_dinc provides per capita disposible income at the state level, including slope of 12-month prior least square fit 
df_dinc = pd.read_csv("C:/Users/Steve/Georgetown/Capstone/data/Monthly_Data_12m_prior/F_percap_dinc.csv", skiprows=0)

In [69]:
df_dinc.tail(n=3)

,addr_state,issue_d,pdi,pdi_LSFT
5607,WY,05/15/15,48667.15833,52.743878
5608,WY,06/15/15,48705.57917,43.476026
5609,WY,07/15/15,48744.00000,38.420833


In [70]:
# df_gdp provides per capita GDP at the state level, including slope of 12-month prior least square fit
df_gdp = pd.read_csv("C:/Users/Steve/Georgetown/Capstone/data/Monthly_Data_12m_prior/F_GDP_state_monthly.csv", skiprows=0)

In [71]:
df_gdp.tail(n=3)

,addr_state,issue_d,gdp,gdp_LSFT
6117,WY,02/15/16,62703.22685,-351.773060
6118,WY,03/15/16,62178.29328,-423.031172
6119,WY,04/15/16,61653.35971,-477.392289


In [72]:
# df_spread provides the spread between the 10-year and 3-month T-bill rates
df_spread = pd.read_csv("C:/Users/Steve/Georgetown/Capstone/data/Monthly_Data_12m_prior/F_10y_3m_spread.csv", skiprows=0)

In [73]:
df_spread.tail(n=3)

,issue_d,spread
115,04/15/16,1.5935
116,05/15/16,1.5895
117,06/15/16,1.3790


In [74]:
# df_tbill provides 3-year federal T-bill rates on a monthly basis, including slope of 12 onth prior least square fit
df_tbill = pd.read_csv("C:/Users/Steve/Georgetown/Capstone/data/Monthly_Data_12m_prior/F_3yr_Tbills.csv", skiprows=0)

In [75]:
df_tbill.tail(n=3)

,issue_d,3yr_rate,3yr_LSFT
119,04/15/16,0.0092,0.000052
120,05/15/16,0.0097,-0.000030
121,06/15/16,0.0086,-0.000112


In [76]:
# df_cc provides average credit card interest rates on a monthly basis
df_cc = pd.read_csv("C:/Users/Steve/Georgetown/Capstone/data/Monthly_Data_12m_prior/F_credit_card_interest_monthly.csv", skiprows=0)

In [77]:
df_cc.tail(n=3)

,issue_d,cc_rate
100,06/15/15,0.1202
101,07/15/15,0.1206
102,08/15/15,0.1210


In [78]:
df = df.merge(df_unemp, on = ('addr_state', 'issue_d'), how = 'left')

In [79]:
df = df.merge(df_dinc, on = ('addr_state', 'issue_d'), how = 'left')
df = df.merge(df_gdp, on = ('addr_state', 'issue_d'), how = 'left')
df = df.merge(df_spread, on = ('issue_d'), how = 'left')
df = df.merge(df_tbill, on = ('issue_d'), how = 'left')
df = df.merge(df_cc, on = ('issue_d'), how = 'left')

In [80]:
# Useful to reset id's at this point
df = df.reset_index(drop=True)
df.tail(n=5)

,funded_amnt,term,int_rate,installment,grade,sub_grade,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,purpose,addr_state,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_rec_prncp,net_default_amnt,unemp,unemp_LSFT,pdi,pdi_LSFT,gdp,gdp_LSFT,spread,3yr_rate,3yr_LSFT,cc_rate
84790,6500,36,9.64,208.66,B,B4,0.5,RENT,20000.0,Not Verified,06/15/07,Default,other,CT,10.00,0.0,1.0,9.0,0.0,0,56.4,1791.32,4708.68,0.045,-0.000071,46784.25750,176.692500,66628.99049,403.740111,0.226,0.05,-0.000207,0.135033
84791,3500,36,10.28,113.39,C,C1,0.5,RENT,180000.0,Not Verified,06/15/07,Paid,other,NY,10.00,0.0,1.0,9.0,0.0,0,56.4,3500.00,0.00,0.045,-0.000017,39438.02667,185.783333,58336.72401,247.415786,0.226,0.05,-0.000207,0.135033
84792,1000,36,9.64,32.11,B,B4,0.5,RENT,12000.0,Not Verified,06/15/07,Paid,other,MA,10.00,0.0,1.0,9.0,0.0,0,56.4,1000.00,0.00,0.048,-0.000165,42169.13167,99.208333,58683.34246,272.211178,0.226,0.05,-0.000207,0.135033
84793,2525,36,9.33,80.69,B,B3,0.5,RENT,110000.0,Not Verified,06/15/07,Paid,other,NY,10.00,0.0,1.0,9.0,0.0,0,56.4,2525.00,0.00,0.045,-0.000017,39438.02667,185.783333,58336.72401,247.415786,0.226,0.05,-0.000207,0.135033
84794,5000,36,7.75,156.11,A,A3,15,MORTGAGE,70000.0,Not Verified,06/15/07,Paid,other,CT,8.81,0.0,1.0,9.0,0.0,0,56.4,5000.00,0.00,0.045,-0.000071,46784.25750,176.692500,66628.99049,403.740111,0.226,0.05,-0.000207,0.135033


In [81]:
# To provide additional relevent features for machine learning, let's create the following features
# rFunAmt_AI -- ratio of funded_amnt to annual_inc
# rTotPay_AI -- ratio of total payments to annual_inc
# rRevBal_AI -- ratio of revol_bal to annual_inc

# Let's create new columns for the above
df['rFunAmt_AI'] = 0
df['rTotPay_AI'] = 0
df['rRevBal_AI'] = 0

In [82]:
df['rFunAmt_AI'] = (df['funded_amnt'])/(df['annual_inc'])
df['rTotPay_AI'] = (df['installment'])*(df['term'])/(df['annual_inc'])
df['rRevBal_AI'] = (df['revol_bal'])/(df['annual_inc'])

In [83]:
df.head(n=3)

,funded_amnt,term,int_rate,installment,grade,sub_grade,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,purpose,addr_state,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_rec_prncp,net_default_amnt,unemp,unemp_LSFT,pdi,pdi_LSFT,gdp,gdp_LSFT,spread,3yr_rate,3yr_LSFT,cc_rate,rFunAmt_AI,rTotPay_AI,rRevBal_AI
0,22000,36,17.27,787.32,C,C5,15,RENT,49000.0,Verified,01/15/13,Paid,debt_consolidation,WA,29.78,0.0,0.0,10.0,0.0,14681,75.7,22000.0,0.0,0.083,-0.001456,42861.355,81.672500,56989.90300,61.285997,1.808,0.0039,-0.000048,0.119133,0.448980,0.578439,0.299612
1,4750,36,19.05,174.24,D,D4,4,RENT,42000.0,Not Verified,01/15/13,Paid,other,NY,19.14,0.0,1.0,12.0,0.0,8890,42.1,4750.0,0.0,0.091,-0.000742,45370.920,38.340417,66998.43403,297.043181,1.808,0.0039,-0.000048,0.119133,0.113095,0.149349,0.211667
2,2000,36,19.72,74.05,D,D5,15,OWN,45000.0,Not Verified,01/15/13,Paid,other,VT,3.72,0.0,0.0,3.0,0.0,3986,43.3,2000.0,0.0,0.053,-0.000907,40217.540,68.987500,45040.88048,-14.385545,1.808,0.0039,-0.000048,0.119133,0.044444,0.05924,0.088578


In [84]:
# Let's save dfe dataframe
df.to_csv ('C:/Users/Steve/Georgetown/Capstone/data/Archived_data_files/dfsave2.csv', sep=',')

In [85]:
# Now that we added state level economic data we can delete 'addr_state'
del df['addr_state']

In [86]:
# Save the dataframe with categorical data
df.to_csv ('C:/Users/Steve/Georgetown/Capstone/data/Archived_data_files/df_cat.csv', sep=',')

In [87]:
df.head(n=5)

,funded_amnt,term,int_rate,installment,grade,sub_grade,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,purpose,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_rec_prncp,net_default_amnt,unemp,unemp_LSFT,pdi,pdi_LSFT,gdp,gdp_LSFT,spread,3yr_rate,3yr_LSFT,cc_rate,rFunAmt_AI,rTotPay_AI,rRevBal_AI
0,22000,36,17.27,787.32,C,C5,15,RENT,49000.0,Verified,01/15/13,Paid,debt_consolidation,29.78,0.0,0.0,10.0,0.0,14681,75.7,22000.0,0.0,0.083,-0.001456,42861.355,81.672500,56989.90300,61.285997,1.808,0.0039,-0.000048,0.119133,0.448980,0.578439,0.299612
1,4750,36,19.05,174.24,D,D4,4,RENT,42000.0,Not Verified,01/15/13,Paid,other,19.14,0.0,1.0,12.0,0.0,8890,42.1,4750.0,0.0,0.091,-0.000742,45370.920,38.340417,66998.43403,297.043181,1.808,0.0039,-0.000048,0.119133,0.113095,0.149349,0.211667
2,2000,36,19.72,74.05,D,D5,15,OWN,45000.0,Not Verified,01/15/13,Paid,other,3.72,0.0,0.0,3.0,0.0,3986,43.3,2000.0,0.0,0.053,-0.000907,40217.540,68.987500,45040.88048,-14.385545,1.808,0.0039,-0.000048,0.119133,0.044444,0.05924,0.088578
3,5000,36,19.05,183.41,D,D4,15,RENT,48000.0,Not Verified,01/15/13,Paid,wedding,3.00,0.0,0.0,2.0,0.0,736,92.0,5000.0,0.0,0.102,-0.001368,41645.985,90.985833,56505.74012,173.826094,1.808,0.0039,-0.000048,0.119133,0.104167,0.137557,0.015333
4,2500,36,21.49,94.82,E,E3,7,RENT,38000.0,Not Verified,01/15/13,Paid,medical,11.02,0.0,0.0,5.0,1.0,1142,40.8,2500.0,0.0,0.104,-0.000049,40540.965,57.426250,55578.04489,35.520356,1.808,0.0039,-0.000048,0.119133,0.065789,0.0898295,0.030053


In [89]:
df['rAPay_AI'] = (df['installment'])*(12)/(df['annual_inc'])

In [91]:
# Save the dataframe with categorical data
df.to_csv ('C:/Users/Steve/Georgetown/Capstone/data/Archived_data_files/df_cat.csv', sep=',')

In [88]:
# Determine which of the above variables are highly correlated and not independent
pd.options.display.max_rows = 150
cor = df.corr()
cor.loc[:,:] = np.tril(cor, k=-1) # below main lower triangle of an array
cor = cor.stack()
cor[(cor > 0.55) | (cor < -0.55)]

installment      funded_amnt    0.986764
total_rec_prncp  funded_amnt    0.909908
                 installment    0.895119
gdp              pdi            0.769962
gdp_LSFT         pdi_LSFT       0.555633
3yr_rate         unemp_LSFT     0.565671
cc_rate          spread         0.864647
                 3yr_rate       0.637295
rFunAmt_AI       funded_amnt    0.564481
                 installment    0.558195
rRevBal_AI       revol_bal      0.697798
dtype: float64